# Настройки

In [1]:
!cd /content
!pip install torch torchvision matplotlib
!git clone https://github.com/MariaDys/multi12 сvrp1
!pip install -r /content/сvrp1/requirements.txt

fatal: destination path 'сvrp1' already exists and is not an empty directory.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall timm
!pip install timm==0.6.13
import timm
# model = timm.create_model('efficientnet_b0', pretrained=False)
# print("timm работает корректно")

Found existing installation: timm 0.6.12
Uninstalling timm-0.6.12:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/timm-0.6.12.dist-info/*
    /usr/local/lib/python3.11/dist-packages/timm/*
Proceed (Y/n)? Y
  Successfully uninstalled timm-0.6.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 15.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
segmentation-models-pytorch 0.3.2 requires timm==0.6.12, but you have timm 0.6.13 which is incompatible.


In [ ]:
!wget -O model_weights.pth https://0xdata-public.s3.amazonaws.com/beluga/wildfire/models/forest_ws2_60d_r8_all.pth

--2025-04-15 15:35:43--  https://0xdata-public.s3.amazonaws.com/beluga/wildfire/models/forest_ws2_60d_r8_all.pth
Resolving 0xdata-public.s3.amazonaws.com (0xdata-public.s3.amazonaws.com)... 3.5.29.194, 3.5.29.31, 3.5.29.197, ...
Connecting to 0xdata-public.s3.amazonaws.com (0xdata-public.s3.amazonaws.com)|3.5.29.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51110127 (49M) [binary/octet-stream]
Saving to: ‘model_weights.pth’

model_weights.pth   100%[===================>]  48.74M  36.5MB/s    in 1.3s    

2025-04-15 15:35:45 (36.5 MB/s) - ‘model_weights.pth’ saved [51110127/51110127]



In [ ]:
!pip install rasterio
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.0 MB/s eta 0:00:00


# Функции

In [ ]:
from сvrp1.src.configs.forest_ws2_60d_r8_all import cfg
from сvrp1.src.models.forest_model import Net
import torch
import matplotlib.pyplot as plt
from collections import OrderedDict
import rasterio
import numpy as np
import cv2
import os
from glob import glob
from math import ceil

In [ ]:
import os

def get_paths_for_coordinates(coord_str, base_path):
    """
    coord_str: строка с координатами, например '54.62_4.39'
    base_path: путь к распакованной папке samples (например, /tmp/tmpabc123/samples)

    Возвращает словарь с путями:
      - tiff_dir
      - output_dir
      - tiff_path (путь к ground-truth .tiff маске)
    """
    tiff_dir = os.path.join(base_path, coord_str)
    output_dir = os.path.join(tiff_dir, "output_256x256")

    deforestation_files = [
        f for f in os.listdir(tiff_dir)
        if f.startswith("Deforestation") and f.endswith(".tiff")
    ]
    if not deforestation_files:
        raise FileNotFoundError(f"[{coord_str}] Не найден файл, начинающийся на 'Deforestation' в {tiff_dir}")

    tiff_path = os.path.join(tiff_dir, deforestation_files[0])
    return {
        "tiff_dir": tiff_dir,
        "output_dir": output_dir,
        "tiff_path": tiff_path
    }


def compute_metrics(gt, pred):
        """
        gt: ground-truth маска сегментации;
        pred: предсказанная маска сегментации;

        Вычисляет ряд дополнительных метрик для полученного предсказания модели.
        """

        gt_flat = gt.flatten()
        pred_flat = pred.flatten()
        tn, fp, fn, tp = confusion_matrix(gt_flat, pred_flat, labels=[0,1]).ravel()
        metrics = {
            "IoU": tp/(tp+fp+fn) if (tp+fp+fn)>0 else 0,
            "Precision": precision_score(gt_flat, pred_flat),
            "Recall": recall_score(gt_flat, pred_flat),
            "F1 (Dice)": f1_score(gt_flat, pred_flat),
            # "Accuracy": accuracy_score(gt_flat, pred_flat),
        }
        return metrics


def load_tiff(file_path):
  """
  Загрузка одного TIFF-файла
  """
  with rasterio.open(file_path) as src:
    return src.read(1)



def process_folder(coord_input, model, device):
    """
    coord_input: строка с координатами, например '54.62_4.39';
    model: тестируемая модель;
    device: cuda or cpu.

    Применяет к папке с введенными координатами следующие действия:
      - по координатам получает путь (на гугл диске данные лежат в подпапке с именем coord_input
      - масштабирует все tiff-файлы (каналы) в подпапке до 256x256
      - объединяет каналы со спутников landsat8, sentinel1, sentinel2
      - нормализует каналы до диапазона значений [0,1] по min-max нормализации
      - прогоняет через модель, получает предсказание, сравнивает с ground-truth маской по IoU и еще некоторым метрикам
    """

    try:
        paths = get_paths_for_coordinates(coord_input, base_path)
    except Exception as e:
        print(f"[{coord_input}] ошибка получения пути: {e}")
        return None, None
    tiff_dir = paths["tiff_dir"] #подпапка с каналами
    output_dir = paths["output_dir"] #вложенная папка куда уйдут масштабированные до 256x256 каналы
    tiff_path = paths["tiff_path"] #путь к ground-truth
    os.makedirs(output_dir, exist_ok=True)



    tiff_files = glob(os.path.join(tiff_dir, "*.tiff")) # масштабирование tiff-файлов до 256x256
    for file_path in tiff_files:
        file_name = os.path.basename(file_path)
        out_path = os.path.join(output_dir, file_name)
        if os.path.exists(out_path): # если уже существует масштабированное изображение – пропускаем
            continue
        with rasterio.open(file_path) as src:
            data = src.read(1)
            data_resized = cv2.resize(data, (256, 256), interpolation=cv2.INTER_CUBIC)
            with rasterio.open(
                out_path,
                "w",
                driver="GTiff",
                height=data_resized.shape[0],
                width=data_resized.shape[1],
                count=1,
                dtype=data_resized.dtype,
                crs=src.crs,
                transform=src.transform,
            ) as dst:
                dst.write(data_resized, 1)
    print(f"[{coord_input}] файлы масштабированы до 256x256")



    satellite_data = {"landsat8": [], "sentinel1": [], "sentinel2": []}
    for file in sorted(os.listdir(output_dir)):
        if not file.endswith('.tiff') or file.startswith("Deforestation"):
            continue
        file_path = os.path.join(output_dir, file)

        if "Landsat8" in file:
            satellite_data["landsat8"].append(load_tiff(file_path))
        elif "Sentinel1" in file:
            satellite_data["sentinel1"].append(load_tiff(file_path))
        elif "Sentinel2" in file:
            satellite_data["sentinel2"].append(load_tiff(file_path))

    all_data = satellite_data["landsat8"] + satellite_data["sentinel1"] + satellite_data["sentinel2"]
    all_data_array = np.stack(all_data, axis=0) # numpy массив (C, H, W)
    print("Форма исходного массива:", all_data_array.shape)


    # min-max нормализация
    normalized_data_array = []
    for c in range(all_data_array.shape[0]):
        band = all_data_array[c]
        min_val, max_val = band.min(), band.max()
        if min_val == max_val:
            band_norm = np.zeros_like(band)
        else:
            band_norm = (band - min_val) / (max_val - min_val)
        band_norm = np.clip(band_norm, 0, 1)
        normalized_data_array.append(band_norm)
    normalized_array = np.stack(normalized_data_array, axis=0)
    # print(f"[{coord_input}] Форма после нормализации: {normalized_array.shape}")

    input_tensor = torch.tensor(normalized_array, dtype=torch.float32).unsqueeze(0)  # (1, C, 256, 256)
    expanded_tensor = input_tensor.repeat(1, 8, 1, 1).to(device)
    # print(f"[{coord_input}] Форма тензора после повторения каналов: {expanded_tensor.shape}")


    # предсказание модели model_weights.pth
    dummy_mask = torch.zeros_like(expanded_tensor[:, :1, :, :])
    batch = {"input": expanded_tensor, "mask": dummy_mask}
    with torch.no_grad():
        output = model(batch)
        predicted_mask = torch.sigmoid(output["logits"]).squeeze(0).cpu().numpy()


    # ground truth маска
    with rasterio.open(tiff_path) as src:
        gt_mask = src.read(1)
        gt_mask = (gt_mask > 0).astype(np.uint8)
    gt_mask = gt_mask[:256, :256]
    if gt_mask.ndim == 2:
      gt_mask = np.expand_dims(gt_mask, axis=0)


    thresholds = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] #перебор пороговых значений и вычисление IoU для каждого
    iou_values = {}
    for val_tres in thresholds:
        binary_mask = (predicted_mask > val_tres).astype(float)
        intersection = np.logical_and(gt_mask, binary_mask).sum()
        union = np.logical_or(gt_mask, binary_mask).sum()
        iou_val = intersection / union if union > 0 else 0
        iou_values[val_tres] = iou_val
    print(f"[{coord_input}] Threshold:IoU", " | ".join(f"{thr}:{iou_values[thr]:.4f}" for thr in thresholds))
    best_threshold = max(iou_values, key=iou_values.get) #max IoU
    best_iou = iou_values[best_threshold]
    print(f"[{coord_input}] Лучший порог: {best_threshold} с IoU: {best_iou:.4f}")
    binary_mask_best = (predicted_mask > best_threshold).astype(float)

    #другие метрики
    metrics = compute_metrics(gt_mask, binary_mask_best)

    return best_threshold, best_iou, metrics



def add_gaussian_noise(x: torch.Tensor, sigma: float) -> torch.Tensor:
    noise = torch.randn_like(x) * sigma
    return torch.clamp(x + noise, 0.0, 1.0)

def add_sp_noise(x: torch.Tensor, prob: float) -> torch.Tensor:
    mask = torch.rand_like(x)
    x_noisy = x.clone()
    x_noisy[mask < prob/2] = 0.0
    x_noisy[mask > 1 - prob/2] = 1.0
    return x_noisy


import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt

# Thresholds for search (exactly как в process_folder)
thresholds = [0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

def load_patch_tensor(coord, device, base_path):
    paths = get_paths_for_coordinates(coord, base_path)
    out_dir = paths["output_dir"]
    os.makedirs(out_dir, exist_ok=True)
    # Resize code omitted (assume already done by process_folder run)
    bands = [
        load_tiff(os.path.join(out_dir,f))
        for f in sorted(os.listdir(out_dir))
        if f.endswith(".tiff") and not f.startswith("Deforestation")
    ]
    arr = np.stack(bands,0)[:24]                 # KEEP EXACTLY 24 bands
    # Min‑max normalize
    arr = np.stack([(b - b.min())/(b.max()-b.min()) if b.max()>b.min() else np.zeros_like(b) for b in arr],0)
    inp = torch.tensor(arr, dtype=torch.float32).unsqueeze(0).to(device)
    expanded = inp.repeat(1,8,1,1)
    with rasterio.open(paths["tiff_path"]) as src:
        gt = (src.read(1)>0).astype(np.uint8)[:256,:256]
    return expanded, np.expand_dims(gt,0)


def evaluate_best(inp, gt, model):
    batch = {"input": inp, "mask": torch.zeros_like(inp[:, :1])}
    with torch.no_grad():
        pred = torch.sigmoid(model(batch)["logits"])[0].cpu().numpy()
    best_thr, best_iou, best_metrics = 0, -1, None
    for thr in thresholds:
        bin_mask = (pred > thr).astype(float)
        metrics = compute_metrics(gt, bin_mask)
        if metrics["IoU"] > best_iou:
            best_iou, best_metrics, best_thr = metrics["IoU"], metrics, thr
    return best_thr, best_metrics


# Интерфейс 1

In [ ]:
from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
    matthews_corrcoef,
    cohen_kappa_score
)

In [ ]:
def analyze_model(model_file, data_archive, metrics_selected, noise_types, noise_level):
    noise_level = float(noise_level)
    temp_dir = tempfile.mkdtemp()
    archive_path = os.path.join(temp_dir, "samples.zip")

    # Сохраняем и распаковываем архив
    if hasattr(data_archive, "read"):
        archive_bytes = data_archive.read()
    else:
        with open(data_archive, "rb") as f:
            archive_bytes = f.read()
    with open(archive_path, "wb") as f:
        f.write(archive_bytes)
    with zipfile.ZipFile(archive_path, "r") as zip_ref:
        zip_ref.extractall(temp_dir)

    samples_dir = os.path.join(temp_dir, "samples")
    if not os.path.exists(samples_dir):
        samples_dir = temp_dir
    base_path = samples_dir

    patch_dirs = [d for d in os.listdir(samples_dir) if os.path.isdir(os.path.join(samples_dir, d))]

    # Модель
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    from сvrp1.src.configs.forest_ws2_60d_r8_all import cfg
    from сvrp1.src.models.forest_model import Net
    model = Net(cfg).to(device)

    checkpoint = torch.load(model_file.name, map_location=device)
    new_state_dict = OrderedDict(
        (k[7:], v) if k.startswith("module.") else (k, v)
        for k, v in checkpoint["model"].items()
    )
    model.load_state_dict(new_state_dict)
    model.eval()

    rows = []
    for patch in patch_dirs:
        try:
            inp, gt = load_patch_tensor(patch, device, base_path)
        except Exception as e:
            print(f"Ошибка при обработке {patch}: {e}")
            continue

        base_thr, base_metrics = evaluate_best(inp, gt, model)
        base_metrics["Patch"] = patch
        base_metrics["Noise"] = "clean"
        base_metrics["Level"] = 0.0
        rows.append(base_metrics)

        for noise_type in noise_types:
            if noise_type == "gaussian":
                noisy_inp = add_gaussian_noise(inp.clone(), sigma=noise_level)
            elif noise_type == "salt_pepper":
                noisy_inp = add_sp_noise(inp.clone(), prob=noise_level)
            else:
                continue

            _, noisy_metrics = evaluate_best(noisy_inp, gt, model)
            delta = {f"Δ_{k}": noisy_metrics[k] - base_metrics.get(k, 0) for k in noisy_metrics}
            noisy_metrics["Patch"] = patch
            noisy_metrics["Noise"] = noise_type
            noisy_metrics["Level"] = noise_level
            noisy_metrics.update(delta)
            rows.append(noisy_metrics)

    df_all = pd.DataFrame(rows).round(3)

    # Фильтрация метрик
    if metrics_selected:
        df_all = df_all[["Patch", "Noise", "Level"] + metrics_selected + [f"Δ_{m}" for m in metrics_selected if f"Δ_{m}" in df_all.columns]]

    # Отдельные таблицы
    df_clean = df_all[df_all["Noise"] == "clean"].reset_index(drop=True)
    df_noisy = df_all[df_all["Noise"] != "clean"].reset_index(drop=True)

    # Средние значения по каждому шуму
    df_avg = df_all.groupby("Noise")[metrics_selected].mean().round(3).reset_index()

    # График
    fig = None
    if "Δ_IoU" in df_all.columns:
        try:
            plt.figure(figsize=(8, 5))
            df_plot = df_all[df_all["Noise"] != "clean"].groupby(["Noise", "Level"])["Δ_IoU"].mean().unstack(0)
            df_plot.plot.bar()
            plt.ylabel("Mean Δ IoU vs baseline")
            plt.title("Noise Impact on IoU")
            plt.tight_layout()
            fig = plt.gcf()
        except Exception as e:
            print("Ошибка при построении графика:", e)

    shutil.rmtree(temp_dir)

    return df_clean, df_noisy, df_avg, fig


In [ ]:
inputs = [
    gr.File(label="Загрузите модель (.pth)"),
    gr.File(label="Загрузите архив с данными (samples.zip)"),
    gr.CheckboxGroup(choices=["IoU", "F1 (Dice)", "Precision", "Recall"], label="Выберите метрики для отображения"),
    gr.CheckboxGroup(choices=["gaussian", "salt_pepper"], label="Выберите типы шума"),
    gr.Slider(minimum=0.0, maximum=0.2, step=0.01, value=0.05, label="Уровень шума"),
]

outputs = [
    gr.Dataframe(label="Метрики на чистых данных"),
    gr.Dataframe(label="Метрики на зашумлённых данных"),
    gr.Dataframe(label="Средние значения по каждому типу данных"),
    gr.Plot(label="Диаграмма влияния шума")
]

gr.Interface(
    fn=analyze_model,
    inputs=inputs,
    outputs=outputs,
    title="Анализ влияния шума на предсказания модели",
    description="Загрузите модель и архив с данными, выберите метрики и типы шума. Получите таблицы и график."
).launch(debug=True)


NameError: name 'gr' is not defined